In [1]:
%matplotlib inline

morange = u'#ff7f0e'
mblue = u'#1f77b4'
mgreen = u'#2ca02c'
mred = u'#d62728'
mpurple = u'#9467bd'

In [2]:
brightest_rband = -24.64
brightest_rband = -np.inf
def apply_um_galacticus_matching_error(um_rest_old, rest_old, um_rest_new):
    """
    """
    mag_r_error = rest_old - um_rest_old
    rest_new = um_rest_new + mag_r_error
    return np.where(rest_new < brightest_rband, brightest_rband, rest_new)

def update_observed_rband(r_obs_old, r_rest_old, r_rest_new):
    """
    """
    dr = r_obs_old - r_rest_old
    return r_rest_new + dr

def update_mag_i_obs(ri_rest_old, mag_i_rest_old, mag_i_obs_old, mag_r_rest_new):
    mag_i_rest_new = mag_r_rest_new - ri_rest_old
    kcorrection_i = mag_i_obs_old - mag_i_rest_old
    mag_i_obs_new = kcorrection_i + mag_i_rest_new
    return mag_i_rest_new, mag_i_obs_new


In [3]:
import os
dirname = "/Volumes/simulation_data_2018/protoDC2_v4"
basename = "v4.1.dust136.1substep.all.aph_reduced.hdf5"

fname = os.path.join(dirname, basename)

from astropy.table import Table
mock = Table.read(fname, path='data')

mock.rename_column('um_restframe_extincted_sdss_abs_magr', 'um_mag_r_rest')
mock.rename_column('um_restframe_extincted_sdss_gr', 'um_gr_rest')
mock.rename_column('um_restframe_extincted_sdss_ri', 'um_ri_rest')
mock['um_mag_g_rest'] = mock['um_gr_rest'] + mock['um_mag_r_rest']
mock['um_mag_i_rest'] = mock['um_mag_r_rest'] - mock['um_ri_rest']
print(mock.keys())

/Users/aphearin/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['um_host_halo_mvir', 'um_obs_sm', 'um_obs_sfr', 'um_redshift', 'um_sfr_percentile', 'um_upid', 'um_mpeak', 'um_mag_r_rest', 'um_gr_rest', 'um_ri_rest', 'um_is_on_red_sequence_gr', 'um_is_on_red_sequence_ri', 'isCentral', 'redshift', 'hostHaloMass', 'mag_g_rest', 'mag_g_obs', 'mag_r_rest', 'mag_r_obs', 'mag_i_rest', 'mag_i_obs', 'mag_z_rest', 'mag_z_obs', 'um_mag_g_rest', 'um_mag_i_rest']


In [5]:
from cosmodc2.stellar_mass_remapping import lift_high_mass_mstar

new_mstar = lift_high_mass_mstar(
        mock['um_mpeak'], mock['um_obs_sm'], mock['um_upid'],  mock['redshift'])
mock['new_um_mstar'] = new_mstar
    
from cosmodc2.sdss_colors import magr_monte_carlo
mock['new_um_mag_r_rest'] = magr_monte_carlo(
    mock['new_um_mstar'], mock['um_upid'], mock['redshift'])
mock['new2_um_mag_r_rest'] = magr_monte_carlo(
    mock['new_um_mstar'], mock['um_upid'], mock['redshift'])

#  Estimate the new restframe Galacticus r-band
mock['new_mag_r_rest'] = apply_um_galacticus_matching_error(
    mock['um_mag_r_rest'], mock['mag_r_rest'], mock['new_um_mag_r_rest'])
mock['new2_mag_r_rest'] = apply_um_galacticus_matching_error(
    mock['um_mag_r_rest'], mock['mag_r_rest'], mock['new2_um_mag_r_rest'])

# #  Estimate the new observed Galacticus r-band
mock['new_mag_r_obs'] = update_observed_rband(
    mock['mag_r_obs'], mock['mag_r_rest'], mock['new_um_mag_r_rest'])
mock['new2_mag_r_obs'] = update_observed_rband(
    mock['mag_r_obs'], mock['mag_r_rest'], mock['new2_um_mag_r_rest'])


In [6]:

# #  Estimate the new observed UniverseMachine i-band
_mag_i_rest_new, _mag_i_obs_new = update_mag_i_obs(
    mock['um_ri_rest'], mock['um_mag_i_rest'], 
    mock['mag_i_obs'], mock['new_um_mag_r_rest'])
mock['new_mag_i_obs'] = _mag_i_obs_new
mock['new_mag_i_rest'] = _mag_i_rest_new

_mag_i_rest_new2, _mag_i_obs_new2 = update_mag_i_obs(
    mock['um_ri_rest'], mock['um_mag_i_rest'], 
    mock['mag_i_obs'], mock['new2_um_mag_r_rest'])
mock['new2_mag_i_obs'] = _mag_i_obs_new2
mock['new2_mag_i_rest'] = _mag_i_rest_new2